In [11]:
import pandas as pd
from gurobipy import GRB
import gurobipy as gb

/var/folders/jh/lwh9h51151s6_693cw5_28cr0000gn/T/ipykernel_24522/1165626401.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [16]:
# Path to your Excel file
file_path = 'path_to_your_excel_file.xlsx'

# To read all sheets into a dictionary of DataFrames
data = pd.read_excel("/Users/justinsheng/Downloads/A1 combine.xlsx", sheet_name=None)


In [18]:
# Data extraction from the provided dataframes
# Direct production capacity
direct_capacity = data['Capacity_for_Direct_Production_'].set_index('ProductionFacility')['Capacity'].to_dict()
# Transshipment production capacity
transship_capacity = data['Capacity_for_Transship_Producti'].set_index('ProductionFacility')['Capacity'].to_dict()
# Distribution center capacities
dist_center_capacity = data['Capacity_for_Transship_Distribu'].set_index('TransshipmentHub')['Capacity'].to_dict()
# Direct shipment costs
cost_direct = data['Cost_Production_to_Refinement'].set_index(['ProductionFacility', 'RefinementCenter'])['Cost'].to_dict()
# Transshipment costs
cost_transship = data['Cost_Transshipment_to_Refinemen'].set_index(['TransshipmentHub', 'RefinementCenter'])['Cost'].to_dict()
# Demand at refinement centers
demand = data['Refinement_Demand'].set_index('RefinementCenter')['Demand'].to_dict()

In [19]:
# Create the optimization model
model = gb.Model("Can2Oil Optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [24]:
# Create decision variables for direct shipments
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct_Shipment")

# Create decision variables for transshipment
y = model.addVars(15, 2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment")

In [25]:
# The objective function
direct_objective = gb.quicksum(cost_direct[i+1, j+1]*x[i, j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(cost_transship[j+1, k+1]*y[i, j, k] for i in range(15) for j in range(2) for k in range(5))
model.setObjective(direct_objective + trans_objective, GRB.MINIMIZE)

In [28]:
# Add supply constraints for direct shipments
for i in range(25):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_capacity[i+1], name=f"Direct_Supply_Constraint_{i+1}")

# Add supply constraints for transshipment production
for i in range(15):
    model.addConstr(gb.quicksum(y[i, j, k] for j in range(2) for k in range(5)) <= transship_capacity[i+1], name=f"Transship_Supply_Constraint_{i+1}")

# Add capacity constraints for distribution centers
for j in range(2):
    model.addConstr(gb.quicksum(y[i, j, k] for i in range(15) for k in range(5)) <= dist_center_capacity[j+1], name=f"Dist_Center_Capacity_{j+1}")

# Add demand constraints
for k in range(5):
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(y[i, j, k] for i in range(15) for j in range(2)) >= demand[k+1], name=f"Demand_Constraint_{k+1}")

In [29]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 122 rows, 550 columns and 1075 nonzeros
Model fingerprint: 0xb87f372a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 75 rows and 275 columns
Presolve time: 0.01s
Presolved: 47 rows, 275 columns, 700 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.091000e+03   0.000000e+00      0s
      65    2.2094007e+04   0.000000e+00   0.000000e+00      0s

Solved in 65 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.209400717e+04


In [30]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    print(f"Total Cost: {model.objVal}")
    print("Direct Shipment Quantities:")
    for i in range(25):
        for j in range(5):
            if x[i, j].x > 0:
                print(f"From Production Facility {i+1} to Refinement Center {j+1}: {x[i, j].x} million pounds")

    print("\nTransshipment Quantities:")
    for i in range(15):
        for j in range(2):
            for k in range(5):
                if y[i, j, k].x > 0:
                    print(f"From Production Facility {i+26} through Transshipment Hub {j+1} to Refinement Center {k+1}: {y[i, j, k].x} million pounds")
else:
    print("Optimal solution not found.")

Optimal solution found.
Total Cost: 22094.007167574564
Direct Shipment Quantities:
From Production Facility 1 to Refinement Center 4: 462.0 million pounds
From Production Facility 2 to Refinement Center 2: 103.0 million pounds
From Production Facility 3 to Refinement Center 3: 460.0 million pounds
From Production Facility 5 to Refinement Center 4: 86.0 million pounds
From Production Facility 6 to Refinement Center 2: 217.0 million pounds
From Production Facility 8 to Refinement Center 5: 521.0 million pounds
From Production Facility 9 to Refinement Center 5: 548.0 million pounds
From Production Facility 11 to Refinement Center 5: 354.0 million pounds
From Production Facility 12 to Refinement Center 1: 7.0 million pounds
From Production Facility 12 to Refinement Center 3: 404.0 million pounds
From Production Facility 13 to Refinement Center 1: 104.0 million pounds
From Production Facility 14 to Refinement Center 5: 155.0 million pounds
From Production Facility 15 to Refinement Center 4: